In [34]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.model_selection import train_test_split
import torch
import torchtext
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
# from imblearn.over_sampling import SMOTE
import random
import time

In [35]:
## General Settings

In [36]:
RANDOM_SEED = 123
torch.manual_seed(RANDOM_SEED)

LEARNING_RATE = 0.005
BATCH_SIZE = 100
NUM_EPOCHS = 15
DEVICE = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

EMBEDDING_DIM = 100
HIDDEN_DIM = 256
NUM_CLASSES = 7

In [78]:
df = pd.read_csv('../data/livros.csv')
df

,Titulo,Genero
0,Iniciação ao Estudo da Administração,Administracao
1,Iniciação a Administração geral,Administracao
2,Iniciação a Administração de pessoal,Administracao
3,Administração de Materiais,Administracao
4,Gestão Ambiental na Empresa,Administracao
5,50 Casos Reais de Administração,Administracao
6,Administrando para o Futuro (os anos 90 e a vi...,Administracao
7,Prática da Administração de empresas,Administracao
8,Administração: introdução ao estudo,Administracao
9,Trazendo os princípios de Deming da sala de au...,Administracao


In [38]:
pd.set_option('display.max_rows', None)

In [39]:
del df

In [40]:
## Prepare Dataset with Torchtext

In [41]:
### Defining the feature processing

TEXT = torchtext.data.Field(
    tokenize='spacy', # default splits on whitespace
    tokenizer_language='pt_core_news_sm'
)

### Defining the label processing

LABEL = torchtext.data.LabelField(dtype=torch.long)

In [42]:
fields = [('Titulo', TEXT), ('Genero', LABEL)]

dataset = torchtext.data.TabularDataset(
    path='../data/livros.csv', format='csv',
    skip_header=True, fields=fields)

In [43]:
## Split Dataset into Train/Test

In [44]:
train_data, test_data = dataset.split(
    split_ratio=[0.8, 0.2],
    random_state=random.seed(RANDOM_SEED))

print(f'Num Train: {len(train_data)}')
print(f'Num Test: {len(test_data)}')

Num Train: 930
Num Test: 233


In [45]:
for i in range(0,233):
    print(vars(test_data[i]))

{'Titulo': ['História', 'para', 'o', 'ensino'], 'Genero': 'Historia'}
{'Titulo': ['Introdução', 'àdministração', 'de', 'empresas'], 'Genero': 'Administracao'}
{'Titulo': ['Introdução', 'à', 'Informática'], 'Genero': 'Matematica'}
{'Titulo': ['A', 'Ciência', 'da', 'Biologia'], 'Genero': 'Biologia'}
{'Titulo': ['Bandeirantes'], 'Genero': 'Historia'}
{'Titulo': ['O', 'último', 'teorema', 'de', 'Fermat', 'História', 'do', 'enigma', 'que', 'confudiu', 'as', 'maiores', 'mentes', 'do', 'mundo', '...'], 'Genero': 'Matematica'}
{'Titulo': ['Conjuntos', 'relações', 'funções', 'inequações'], 'Genero': 'Matematica'}
{'Titulo': ['national', 'Gallery', 'of', 'Art', '(', 'Washington', ')'], 'Genero': 'Artes'}
{'Titulo': ['Minimalismo'], 'Genero': 'Artes'}
{'Titulo': ['A', 'saúde', 'do', 'Brasileiro'], 'Genero': 'Historia'}
{'Titulo': ['matemática', 'Comercial', 'financeira'], 'Genero': 'Matematica'}
{'Titulo': ['Os', 'Egípcios'], 'Genero': 'Historia'}
{'Titulo': ['Biologia', '3', '-', 'Genética', '-'

In [46]:
print(vars(test_data.examples[0]))

{'Titulo': ['História', 'para', 'o', 'ensino'], 'Genero': 'Historia'}


In [47]:
## Build Vocabulary

In [48]:
TEXT.build_vocab(train_data)
LABEL.build_vocab(train_data)

print(f'Vocabulary size: {len(TEXT.vocab)}')
print(f'Number of classes: {len(LABEL.vocab)}')

Vocabulary size: 1629
Number of classes: 7


In [49]:
print(TEXT.vocab.freqs.most_common(20))

[('e', 237), (':', 226), ('de', 203), ('da', 162), ('do', 120), ('História', 107), ('A', 98), ('Brasil', 91), ('-', 84), ('a', 79), ('O', 78), ('Matemática', 61), ('o', 54), ('Geografia', 51), ('Biologia', 50), ('Geral', 39), ('no', 38), ('Administração', 37), ('(', 31), (')', 31)]


In [50]:
print(TEXT.vocab.itos[:10]) # itos = integer-to-string

['<unk>', '<pad>', 'e', ':', 'de', 'da', 'do', 'História', 'A', 'Brasil']


In [51]:
print(LABEL.vocab.stoi)

defaultdict(None, {'Historia': 0, 'Administracao': 1, 'Geografia': 2, 'Biologia': 3, 'Matematica': 4, 'Artes': 5, 'Literatura': 6})


In [52]:
LABEL.vocab.freqs

Counter({'Historia': 350,
         'Administracao': 134,
         'Geografia': 116,
         'Biologia': 114,
         'Matematica': 104,
         'Artes': 70,
         'Literatura': 42})

In [53]:
## Define Data Loaders

In [54]:
train_loader ,test_loader = \
        torchtext.data.BucketIterator.splits(
        (train_data, test_data),
         batch_size=BATCH_SIZE,
         sort_within_batch=False,
         sort_key=lambda x: len(x.Titulo),
         device=DEVICE
    )

In [55]:
for i in enumerate(test_loader):
    print(i)

(0, 
[torchtext.data.batch.Batch of size 100]
	[.Titulo]:[torch.LongTensor of size 4x100]
	[.Genero]:[torch.LongTensor of size 100])
(1, 
[torchtext.data.batch.Batch of size 100]
	[.Titulo]:[torch.LongTensor of size 8x100]
	[.Genero]:[torch.LongTensor of size 100])
(2, 
[torchtext.data.batch.Batch of size 33]
	[.Titulo]:[torch.LongTensor of size 22x33]
	[.Genero]:[torch.LongTensor of size 33])


In [56]:
print('Train')
for batch in train_loader:
    print(f'Text matrix size: {batch.Titulo.size()}')
    print(f'Target vector size: {batch.Genero.size()}')
    break
    
print('\nTest:')
for batch in test_loader:
    print(f'Text matrix size: {batch.Titulo.size()}')
    print(f'Target vector size: {batch.Genero.size()}')
    break

Train
Text matrix size: torch.Size([14, 100])
Target vector size: torch.Size([100])

Test:
Text matrix size: torch.Size([4, 100])
Target vector size: torch.Size([100])


In [57]:
## Model

In [67]:
class RNN(torch.nn.Module):
    
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()

        self.embedding = torch.nn.Embedding(input_dim, embedding_dim)
        #self.rnn = torch.nn.RNN(embedding_dim,
        #                        hidden_dim,
        #                        nonlinearity='relu')
        self.rnn = torch.nn.GRU(embedding_dim,
                                 hidden_dim)        
        
        self.fc = torch.nn.Linear(hidden_dim, output_dim)
        

    def forward(self, text):
        # text dim: [sentence length, batch size]

        embedded = self.embedding(text)
        # embedded dim: [sentence length, batch size, embedding dim]

        if isinstance(self.rnn, torch.nn.RNN):
            output, hidden = self.rnn(embedded)
        elif isinstance(self.rnn, torch.nn.GRU):
            output, hidden = self.rnn(embedded)

        # output dim: [sentence length, batch size, hidden dim]
        # hidden dim: [1, batch size, hidden dim]

        hidden.squeeze_(0)
        # hidden dim: [batch size, hidden dim]

        output = self.fc(hidden)
        return output

In [68]:
torch.manual_seed(RANDOM_SEED)
model = RNN(input_dim=len(TEXT.vocab),
            embedding_dim=EMBEDDING_DIM,
            hidden_dim=HIDDEN_DIM,
            output_dim=NUM_CLASSES # could use 1 for binary classification
)

model = model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
print(model)

RNN(
  (embedding): Embedding(1629, 100)
  (rnn): GRU(100, 256)
  (fc): Linear(in_features=256, out_features=7, bias=True)
)


In [69]:
def compute_accuracy(model, data_loader, device):

    with torch.no_grad():

        correct_pred, num_examples = 0, 0

        for i, (features, targets) in enumerate(data_loader):

            features = features.to(device)
            targets = targets.float().to(device)

            logits = model(features)
            _, predicted_labels = torch.max(logits, 1)

            num_examples += targets.size(0)
            correct_pred += (predicted_labels == targets).sum()
            
    return correct_pred.float()/num_examples * 100

In [70]:
start_time = time.time()

for epoch in range(NUM_EPOCHS):
    model.train()
    for batch_idx, batch_data in enumerate(train_loader):
        
        text = batch_data.Titulo.to(DEVICE)
        labels = batch_data.Genero.to(DEVICE)

        ### FORWARD AND BACK PROP
        logits = model(text)
        loss = F.cross_entropy(logits, labels)
        optimizer.zero_grad()
        
        loss.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        ### LOGGING
        if not batch_idx % 50:
            print (f'Epoch: {epoch+1:03d}/{NUM_EPOCHS:03d} | '
                   f'Batch {batch_idx:03d}/{len(train_loader):03d} | '
                   f'Loss: {loss:.4f}')

    with torch.set_grad_enabled(False):
        print(f'training accuracy: 'f'{compute_accuracy(model, train_loader, DEVICE):.2f}%')
        
    print(f'Time elapsed: {(time.time() - start_time)/60:.2f} min')
    
print(f'Total Training Time: {(time.time() - start_time)/60:.2f} min')

print(f'Test accuracy: {compute_accuracy(model, test_loader, DEVICE):.2f}%')

Epoch: 001/015 | Batch 000/010 | Loss: 1.9717
training accuracy: 37.96%
Time elapsed: 0.02 min
Epoch: 002/015 | Batch 000/010 | Loss: 1.7167
training accuracy: 48.39%
Time elapsed: 0.04 min
Epoch: 003/015 | Batch 000/010 | Loss: 1.4809
training accuracy: 73.98%
Time elapsed: 0.06 min
Epoch: 004/015 | Batch 000/010 | Loss: 0.7112
training accuracy: 86.88%
Time elapsed: 0.08 min
Epoch: 005/015 | Batch 000/010 | Loss: 0.4600
training accuracy: 94.84%
Time elapsed: 0.10 min
Epoch: 006/015 | Batch 000/010 | Loss: 0.2467
training accuracy: 98.06%
Time elapsed: 0.12 min
Epoch: 007/015 | Batch 000/010 | Loss: 0.2352
training accuracy: 99.68%
Time elapsed: 0.14 min
Epoch: 008/015 | Batch 000/010 | Loss: 0.0406
training accuracy: 99.68%
Time elapsed: 0.15 min
Epoch: 009/015 | Batch 000/010 | Loss: 0.0522
training accuracy: 100.00%
Time elapsed: 0.17 min
Epoch: 010/015 | Batch 000/010 | Loss: 0.0042
training accuracy: 100.00%
Time elapsed: 0.18 min
Epoch: 011/015 | Batch 000/010 | Loss: 0.0041
tr

In [72]:
import spacy


nlp = spacy.blank("pt")

def predict(model, sentence):

    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(DEVICE)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.nn.functional.softmax(model(tensor), dim=1)
    return prediction


a = predict(model, "uma histórica viagem que mudou o século")

In [77]:
a.detach().numpy().round(3)

array([[0.776, 0.17 , 0.   , 0.01 , 0.002, 0.018, 0.024]], dtype=float32)